In [ ]:

!pip install transformers
from typing import Tuple, List
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, TextDataset, DataCollatorForLanguageModeling,
                          AutoModelForCausalLM,
                          Trainer, TrainingArguments,
                          TextGenerationPipeline, pipeline)
from torch.cuda import empty_cache
import os
import torch
input_path       : str = "/content/drive/MyDrive/datatrue"
train_path       : str = "./train.txt"
test_path        : str = "./test.txt"
output_dir       : str = "./output" 
split_token      : str = "<|endoftext|>"
model_name       : str = "sberbank-ai/rugpt3medium_based_on_gpt2"
input_block_size : int = 128
train_batch_size : int = 6
eval_batch_size  : int = 16
epochs_count     : int = 5
eval_every       : int = 500
save_every       : int = 2500
warmup_steps     : int = 50

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:

def get_test_train_text() -> Tuple[List[str], List[str]]:
    data = []
    for filename in os.listdir(input_path):
        f = os.path.join(input_path, filename)
        if os.path.isfile(f):
            with open(f, "r", encoding="utf-8") as file:
                data.append(file.read())
        
    return train_test_split(data, test_size=0.2)

def save_text_file(text : List[str], path : str):
    data = split_token.join(text)
    with open(path, "w", encoding="utf-8") as file:
        file.write(data)
    
train_data, test_data = get_test_train_text()
save_text_file(train_data, train_path)
save_text_file(test_data, test_path)

In [ ]:

def load_dataset(tokenizer : AutoTokenizer) -> Tuple[TextDataset, TextDataset, DataCollatorForLanguageModeling]:
    train_dataset = TextDataset(tokenizer=tokenizer, 
                                file_path=train_path, 
                                block_size=input_block_size)
    test_dataset = TextDataset(tokenizer=tokenizer, 
                               file_path=test_path, 
                               block_size=input_block_size)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                    mlm=False)
    return train_dataset, test_dataset, data_collator

def load_model(path : str = model_name, resize : bool = True) -> Tuple[AutoTokenizer, AutoModelForCausalLM]:
    tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token=split_token, pad_token=split_token)
    PATH = 'config.json'
    model = AutoModelForCausalLM.from_pretrained(path)
    if resize:
        model.resize_token_embeddings(len(tokenizer))
    return tokenizer, model

In [ ]:
# Дообучение.
def create_metrics_computer():
    metrics = {
        "bleu" : load_metric("bleu"),
        "rouge" : load_metric("rouge")
    }
        
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        bleu_results = metrics["bleu"].compute(predictions=predictions, references=labels)
        rouge_results = metrics["rouge"].compute(predictions=predictions, references=labels)
        metrics_results= {
            "bleu" : bleu_results["bleu"],
            "rouge1" : rouge_results["rouge1"].mid.fmeasure,
            "rouge2" : rouge_results["rouge2"].mid.fmeasure,
            "rougeL" : rouge_results["rougeL"].mid.fmeasure,
            "rougeLsum" : rouge_results["rougeLsum"].mid.fmeasure,
        }
        return metrics_results
    
    return compute_metrics

def create_trainer(model : AutoModelForCausalLM,
                   data_collator : DataCollatorForLanguageModeling,
                   train_dataset : TextDataset,
                   test_dataset  : TextDataset) -> Trainer:
    training_args = TrainingArguments(output_dir=output_dir,
                                      overwrite_output_dir=True,
                                      evaluation_strategy="steps",
                                      num_train_epochs=epochs_count,
                                      per_device_train_batch_size=train_batch_size,
                                      per_device_eval_batch_size=eval_batch_size,
                                      logging_steps=eval_every,
                                      eval_steps=eval_every,
                                      save_steps=save_every,
                                      warmup_steps=warmup_steps)
    trainer = Trainer(model=model,
                      args=training_args,
                      data_collator=data_collator,
                      train_dataset=train_dataset,
                      eval_dataset=test_dataset)
    return trainer


empty_cache()
tokenizer, model = load_model()
train_dataset, test_dataset, data_collator = load_dataset(tokenizer)
trainer = create_trainer(model, data_collator, train_dataset, test_dataset)
trainer.train()


Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14362
  Num Epochs = 5
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 11970


Step,Training Loss,Validation Loss
500,2.255400,2.007337
1000,1.873400,1.803091
1500,1.708900,1.686970
2000,1.583600,1.598986
2500,1.445400,1.525234
3000,1.234600,1.458513
3500,1.203300,1.399154
4000,1.173800,1.352797
4500,1.110400,1.291766
5000,1.016400,1.274544


***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
Saving model checkpoint to ./output/checkpoint-2500
Configuration saved in ./output/checkpoint-2500/config.json
Model weights saved in ./output/checkpoint-2500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
Saving model checkpoint to ./output/checkpoint-5000
Configuration saved in ./output/checkpoint-5000/config.json


Step,Training Loss,Validation Loss
500,2.255400,2.007337
1000,1.873400,1.803091
1500,1.708900,1.686970
2000,1.583600,1.598986
2500,1.445400,1.525234
3000,1.234600,1.458513
3500,1.203300,1.399154
4000,1.173800,1.352797
4500,1.110400,1.291766
5000,1.016400,1.274544


***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
Saving model checkpoint to ./output/checkpoint-7500
Configuration saved in ./output/checkpoint-7500/config.json
Model weights saved in ./output/checkpoint-7500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
***** Running Evaluation *****
  Num examples = 3530
  Batch size = 16
Saving model checkpoint to ./output/checkpoint-10000
Configuration saved in ./output/checkpoint-10000/config.json
Model weights saved in ./output/checkpoint-10000/pytorch_model.bin
**

In [ ]:
# Генерация дообученной моделью.

saved_model_path ="/content/drive/MyDrive/GPT2 model/checkpoint-10000"
tokenizer, model = load_model(saved_model_path, resize=False)
eval_model = TextGenerationPipeline(model=model, 
                                    tokenizer=tokenizer,
                                    batch_size=eval_batch_size,
                                    device=0,
                                    max_length=250)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
starters = ["СОГЛАШЕНИЕ ОБ ИСПОЛЬЗОВАНИИ ФАЙЛОВ КУКИ-ФАЙЛОВ (COOKIE). При использовании веб-сайта: vp43.ru Вы подтверждаете свое согласие на использование Компанией ИП Баширова Анастасия Игоревна cookie-файлов в соответствии с настоящим соглашением в отношении данного типа файлов."]
for starter in starters:
    predictions = eval_model(starter)
    print("--------------------")
    print(predictions[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--------------------
СОГЛАШЕНИЕ ОБ ИСПОЛЬЗОВАНИИ ФАЙЛОВ КУКИ-ФАЙЛОВ (COOKIE). При использовании веб-сайта: vp43.ru Вы подтверждаете свое согласие на использование Компанией ИП Баширова Анастасия Игоревна cookie-файлов в соответствии с настоящим соглашением в отношении данного типа файлов. Если Вы не согласны с тем, чтобы Компания использовала cookie-файлы, то Вы должны соответствующим образом установить настройки Вашего браузера или не использовать настоящий веб-сайт и покинуть его.

Что такое cookie-файлы?
Cookie-файлы – это небольшие текстовые файлы, размещаемые на вашем компьютере веб-сайтами, которые вы посетили. Они широко используются для того, чтобы веб-сайты могли работать вообще или работать эффективнее, а также чтобы предоставлять необходимую информацию владельцам веб-сайта. Использование cookie-файлов – стандартная на данный момент практика для большинства веб-сайтов.

Как мы используем cookie-файлы?
Информация, которую мы собираем с помощью файлов cookie-файлов, помогает на